# This notebook is to test a single batch run in ADAM

In [ ]:
from adam import Batch
from adam import Batches
from adam import BatchRunManager
from adam import PropagationParams
from adam import OpmParams
from adam import ConfigManager
from adam import Projects
from adam import RestRequests
from adam import AuthenticatingRestProxy
import time
import os

This sets up authenticated access to the server. It needs to be done before pretty much everything you want to do with ADAM.

In [ ]:
# Reads your config from a file in current directory. For instructions on setting this up, see config_demo notebook.
config = ConfigManager().get_config(environment='local')
auth_rest = AuthenticatingRestProxy(RestRequests(config['url']), config['token'])

## Example Inputs

In [ ]:
# 6x1 state vector (position [km], velocity [km/s])
# asteroid 101
state_vec3 = [-150874809.2,
              -187234595.3,
              -73785026.7,
              14.64403935,
              -11.75744819,
              -5.583528281]

### Set Parameters

Commented parameters are optional. Uncomment to use.

In [ ]:
propagation_params3 = PropagationParams({
    'start_time': '2001-01-02T01:13:46.620000Z',   # propagation start time in ISO format
    'end_time': '2010-12-01T01:13:46.620000Z',     # propagation end time in ISO format

    'project_uuid': config['workspace'],

     'step_size': 86400,                   # step size (seconds)
     'propagator_uuid': '00000000-0000-0000-0000-000000000003',  # force model: sun, moon, planets, 16 asteroids
#     'description': 'some description'       # description of run
})

opm_params3 = OpmParams({
    'epoch': '2001-01-02T01:13:46.620000Z',
    'state_vector': state_vec3,
})

### Submit and Run Propagation

In [ ]:
batch3 = Batch(propagation_params3, opm_params3)
print("Submitting OPM:")
print(batch3.get_opm_params().generate_opm())

In [ ]:
# Submit and wait until batch run is ready
batches_module = Batches(auth_rest)
BatchRunManager(batches_module, [batch3]).run()

### Get Status and Parts Count

In [ ]:
# Get final status and parts count
parts_count = batch3.get_state_summary().get_parts_count()
print("Final state: %s, part count %s\n" % (batch3.get_calc_state(), parts_count))

### Get Ephemeris of Specified Part

In [ ]:
# Get ephemeris of specified part
part_to_get = 0
eph3 = batch3.get_results().get_parts()[part_to_get].get_ephemeris()
print("Ephemeris:")
print(eph3)

### Get ending state vector

In [ ]:
# Get the end state vector (uncomment to use)
end_state_vector3 = batch3.get_results().get_end_state_vector()
print("State vector at the end of propagation:")
print(end_state_vector3)
print("should be: [62739872.06, 117402282.4, 47549221.51, -31.06961002, 13.64009464, 7.070881807]")